In [107]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras.models import Sequential,load_model
from keras.layers import Dense,LSTM,Embedding,Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk
import string
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.remove('not')
ps = nltk.PorterStemmer()
lm = nltk.WordNetLemmatizer()
from keras.models import load_model
import pickle

In [108]:
from keras.preprocessing import sequence, text
from keras.layers import Bidirectional

In [109]:
embeddings_index = {}
f = open('glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [05:54, 6198.91it/s]


Found 2196016 word vectors.


In [250]:
data1=pd.read_csv("cats1.csv",encoding= 'unicode_escape')
data1=data1.sample(frac=1).reset_index(drop=True)
print(data1.shape)
data1.head()

(4015, 3)


,module output,emotion,label
0,"I probably won't be back, to be honest.",sadness,Negative
1,We waited for thirty minutes to be seated (alt...,sadness,Negative
2,If you read this because its Star Wars try to ...,Neutral,Neutral
3,"The hostel's food was so crap, I was sick",Hate,Negative
4,Do you know how to speak english,Surprise,Positive


In [251]:
data1=data1[['module output','label']]
data1.head()

,module output,label
0,"I probably won't be back, to be honest.",Negative
1,We waited for thirty minutes to be seated (alt...,Negative
2,If you read this because its Star Wars try to ...,Neutral
3,"The hostel's food was so crap, I was sick",Negative
4,Do you know how to speak english,Positive


In [252]:
data2 = pd.read_csv('Tweets.csv')
data2 = data2.sample(frac=1).reset_index(drop=True)
print(data2.shape)
data2.head()

(14640, 15)


,tweet_id,label,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,module output,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.702750e+17,positive,0.6749,NaN,0.0000,American,NaN,susqhb,NaN,0,@AmericanAir None of the #LAX flights into #DF...,NaN,2015-02-24 09:33:24 -0800,Dallas via NYC via the OC,Central Time (US & Canada)
1,5.684920e+17,negative,1.0000,Can't Tell,1.0000,Southwest,NaN,vizualpupil,NaN,0,"@SouthwestAir thanks a lot for sending a ""spec...","[29.98473682, -90.25978334]",2015-02-19 11:26:45 -0800,"Chicago--Savannah--Atlanta, GA",Eastern Time (US & Canada)
2,5.699430e+17,positive,1.0000,NaN,NaN,Virgin America,NaN,mrmichaellay,NaN,0,"@VirginAmerica not worried, it's been a great ...","[36.08457854, -115.13780136]",2015-02-23 11:32:31 -0800,Floridian from Cincinnati,Eastern Time (US & Canada)
3,5.680720e+17,neutral,0.6366,NaN,NaN,United,NaN,rabbijfranklin,NaN,0,"@united yes, would love an upgrade or voucher,...",NaN,2015-02-18 07:38:46 -0800,Needham,Atlantic Time (Canada)
4,5.682720e+17,negative,1.0000,Late Flight,0.6813,Southwest,NaN,erinpeep,NaN,0,@SouthwestAir We've been sitting at the gate f...,"[33.94729537, -118.40196951]",2015-02-18 20:51:03 -0800,"Los Angeles, CA",Pacific Time (US & Canada)


In [253]:
data2 = data2[['module output', 'label']]
data2.head()

,module output,label
0,@AmericanAir None of the #LAX flights into #DF...,positive
1,"@SouthwestAir thanks a lot for sending a ""spec...",negative
2,"@VirginAmerica not worried, it's been a great ...",positive
3,"@united yes, would love an upgrade or voucher,...",neutral
4,@SouthwestAir We've been sitting at the gate f...,negative


In [254]:
data2['module output'] = data2['module output'].str.replace('@VirginAmerica', '')
data2.head()

,module output,label
0,@AmericanAir None of the #LAX flights into #DF...,positive
1,"@SouthwestAir thanks a lot for sending a ""spec...",negative
2,"not worried, it's been a great ride in a new ...",positive
3,"@united yes, would love an upgrade or voucher,...",neutral
4,@SouthwestAir We've been sitting at the gate f...,negative


In [255]:
data2['module output'].apply(lambda x: x.lower()) #transform text to lowercase
data2['module output'] = data2['module output'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
data2['module output'].head()

0    AmericanAir None of the LAX flights into DFW h...
1    SouthwestAir thanks a lot for sending a specia...
2     not worried its been a great ride in a new pl...
3    united yes would love an upgrade or voucher  p...
4    SouthwestAir Weve been sitting at the gate for...
Name: module output, dtype: object

In [256]:
data3=pd.read_csv("cats2.csv",encoding= 'unicode_escape')
data3=data3.sample(frac=1).reset_index(drop=True)
print(data3.shape)
data3.head()

(104, 3)


,module output,Emotion,label
0,"Make work from home mandatory, there will be l...",worry,negative
1,Trump is one who needs caution and compassion ...,worry,neutral
2,There are flight for July 19 and 20 Bangalore....,worry,neutral
3,When everything is owned by the government and...,neutral,neutral
4,Remember how Netaji Subhas Chandra Bose was tr...,sadness,negative


In [257]:
data3=data3[['module output','label']]
data3.head()

,module output,label
0,"Make work from home mandatory, there will be l...",negative
1,Trump is one who needs caution and compassion ...,neutral
2,There are flight for July 19 and 20 Bangalore....,neutral
3,When everything is owned by the government and...,neutral
4,Remember how Netaji Subhas Chandra Bose was tr...,negative


In [258]:
data4=pd.read_csv("cats3.csv",encoding= 'unicode_escape')
data4=data4.sample(frac=1).reset_index(drop=True)
print(data4.shape)
data4.head()

(3466, 2)


,module output,label
0,your system does not recognize spaces in last ...,Negative
1,Chaos in over call for secret vote.,Neutral
2,please take all the rest you needwe love you s...,Negative
3,1 short of two-thirds majority.,Neutral
4,reverse polarisation and more. In special epap...,Neutral


In [259]:
data4=data4[['module output','label']]
data4.head()

,module output,label
0,your system does not recognize spaces in last ...,Negative
1,Chaos in over call for secret vote.,Neutral
2,please take all the rest you needwe love you s...,Negative
3,1 short of two-thirds majority.,Neutral
4,reverse polarisation and more. In special epap...,Neutral


In [260]:
data5=pd.read_csv("not.csv",encoding= 'unicode_escape')
data5=data5.sample(frac=1).reset_index(drop=True)
print(data5.shape)
data5.head()

(29, 2)


,module output,label
0,"I am not sure, if it fixes the Covid issue",neutral
1,I always forget to make a note of the things,neutral
2,"I am pairing this with my iphone, and I could ...",positive
3,get lost and do not show me that face of your'...,negative
4,"is this for real? If not, you have been much t...",negative


In [261]:
data5=data5[['module output','label']]
data5.head()

,module output,label
0,"I am not sure, if it fixes the Covid issue",neutral
1,I always forget to make a note of the things,neutral
2,"I am pairing this with my iphone, and I could ...",positive
3,get lost and do not show me that face of your'...,negative
4,"is this for real? If not, you have been much t...",negative


In [262]:
data6=pd.read_csv("goodbad.csv",encoding= 'unicode_escape')
data6=data6.sample(frac=1).reset_index(drop=True)
print(data6.shape)
data6.head()

(70, 2)


,module output,label
0,hellooooooo!!!!!! we finally get to speak!!! a...,positive
1,"Call me a taxi, please.",neutral
2,I saw someone on the hill with a telescope.,neutral
3,Look at the dog using only one of your eyes.,neutral
4,can some please explain to me how taxes work? ...,negative


In [263]:
data6=data6[['module output','label']]
data6.head()

,module output,label
0,hellooooooo!!!!!! we finally get to speak!!! a...,positive
1,"Call me a taxi, please.",neutral
2,I saw someone on the hill with a telescope.,neutral
3,Look at the dog using only one of your eyes.,neutral
4,can some please explain to me how taxes work? ...,negative


In [264]:
er_x = pd.DataFrame(data5['module output'])
print(er_x)
er_y = pd.DataFrame(data5['label'])
print(er_y)

                                        module output
0          I am not sure, if it fixes the Covid issue
1        I always forget to make a note of the things
2   I am pairing this with my iphone, and I could ...
3   get lost and do not show me that face of your'...
4   is this for real? If not, you have been much t...
5                 I am not going to trust you anymore
6   The day we won the match is the day I will nev...
7   You get extra minutes so that you can carry ou...
8                                          DO NOT BUY
9   It didn't work, people can not hear me when I ...
10                  You are not only good but amazing
11  This show is not only good but the best show ever
12                        The battle is not yet over.
13  im not even arguing about the ship anymore, im...
14  Thats a bad photo shop. The original shows a ...
15                 Jokes on you Im not even american
16                       I am not at all angry at you
17  Thats a bad photo shop.

In [265]:
ar_x = pd.DataFrame(data1['module output'])
print(ar_x)
ar_y = pd.DataFrame(data1['label'])
print(ar_y)

                                          module output
0               I probably won't be back, to be honest.
1     We waited for thirty minutes to be seated (alt...
2     If you read this because its Star Wars try to ...
3             The hostel's food was so crap, I was sick
4                      Do you know how to speak english
5                                          Of any kind!
6                       The noise shield is incrediable
7                                      explained below.
8                        what is the weather like today
9     I have bought this Nokia cell phone a few week...
10                                       Where can I go
11                                    Plumbing app that
12    Be sure to order dessert, even if you need to ...
13                 Can't understand how to book a train
14    Perhaps Joseph Martin wanted to make Harris th...
15                                     IT'S REALLY EASY
16                                            go

In [266]:
br_x = pd.DataFrame(data2['module output'])
print(br_x)
br_y = pd.DataFrame(data2['label'])
print(br_y)

                                           module output
0      AmericanAir None of the LAX flights into DFW h...
1      SouthwestAir thanks a lot for sending a specia...
2       not worried its been a great ride in a new pl...
3      united yes would love an upgrade or voucher  p...
4      SouthwestAir Weve been sitting at the gate for...
5      SouthwestAir Flight 4110 MCO to ISP delayed An...
6      USAirways I have LITERALLY been on hold since ...
7       Flight from BOS gt LAS tomorrow was Cancelled...
8      united fire yr rep who refused to put me on a ...
9      SouthwestAir I was never able to talk so someo...
10     USAirways Flight 830 CLT to Phl I was 1st on l...
11     SouthwestAir FJBFSC  all I need is receipt sho...
12     united I do not see where it talks about milit...
13     USAirways my friends at KPHL were told by your...
14     SouthwestAir Me on one of your planes Thanks f...
15     Stop this madness RT JetBlue Our fleets on fle...
16     USAirways my number is 2

In [267]:
cr_x = pd.DataFrame(data3['module output'])
print(cr_x)
cr_y = pd.DataFrame(data3['label'])
print(cr_y)

                                         module output
0    Make work from home mandatory, there will be l...
1    Trump is one who needs caution and compassion ...
2    There are flight for July 19 and 20 Bangalore....
3    When everything is owned by the government and...
4    Remember how Netaji Subhas Chandra Bose was tr...
5     Majestic who returned normal after the lockdown.
6                                Just retweet no likes
7    THE IMPACT ON DESIGN\r, Naveen G.J., Principal...
8                                      Join this group
9    bangalore Commissioner Bhaskar Rao Reacts On L...
10                                            We will 
11   "...it doesn't matter what the judges think if...
12   We are all tired, but we need to keep going. T...
13     Attendance time \r\nRetweet if you're online \r
14   19/07/2020 05:06\r\r\nAir Temperature: 23.7Â°C...
15   The speed at which it's spreading now, I don't...
16                         Still 12 hrs 30 mins left\r
17   If th

In [268]:
dr_x = pd.DataFrame(data4['module output'])
print(dr_x)
dr_y = pd.DataFrame(data4['label'])
print(dr_y)

                                          module output
0     your system does not recognize spaces in last ...
1                   Chaos in over call for secret vote.
2     please take all the rest you needwe love you s...
3                       1 short of two-thirds majority.
4     reverse polarisation and more. In special epap...
5         Britain is not becoming xenophobic or hostile
6     Appreciate your recent retweets Have a great  ...
7         mastermind of module of arrested by police in
8     mutual: let's talk more!me: for sure!mutual:  ...
9     yes... kind of funny and creepy at the same ti...
10                                           love happy
11    army blocks key highway. And civilians are suf...
12    Palaniswami's victory (and how the dhoti survi...
13                Yeesh unhappy  It's fairly warm here.
14    Cong forced to  adjournments in pre-noon sessi...
15                                            cheeky :D
16                        but surges to runner-u

In [269]:
fr_x = pd.DataFrame(data6['module output'])
print(fr_x)
fr_y = pd.DataFrame(data6['label'])
print(fr_y)

                                        module output
0   hellooooooo!!!!!! we finally get to speak!!! a...
1                             Call me a taxi, please.
2         I saw someone on the hill with a telescope.
3        Look at the dog using only one of your eyes.
4   can some please explain to me how taxes work? ...
5   He read the book his friend had given him last...
6                      They fed her the dog biscuits.
7   where is the world s largest ice sheet located...
8   i love to eat pizza but i have high cholestrol...
9                       We need more humane officers.
10    They saw the girl who disappeared in the store.
11              The dog barked at the cat in the park
12                     Flying planes can be dangerous
13  Tanaka was awarded the 2002 Nobel Prize in Che...
14  friend or foe, in the wake of the confluence, ...
15           He looked professional in his chefs hat
16  ELON I GOT HEAD IN MY TESLA ON AUTO PILOT AND ...
17  The latest character rel

In [270]:
x_feature = pd.concat([ar_x,br_x,cr_x,dr_x,er_x,fr_x],ignore_index=True)
print(x_feature.shape)
y_feature = pd.concat([ar_y,br_y,cr_y,dr_y,er_y,fr_y],ignore_index=True)
print(y_feature.shape)

(22324, 1)
(22324, 1)


In [271]:
def clean_text1(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    # text = "".join([word for word in text if word.isalpha()])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word.isalpha()]
    text = " ".join([lm.lemmatize(word) for word in text if word not in stop])
    return text

In [272]:
x_feature['body']=x_feature['module output'].apply(lambda x:clean_text1(x))
x_feature.head()

,module output,body
0,"I probably won't be back, to be honest.",probably wont back honest
1,We waited for thirty minutes to be seated (alt...,waited thirty minute seated although vacant ta...
2,If you read this because its Star Wars try to ...,read star war try experiment borrow one
3,"The hostel's food was so crap, I was sick",hostel food crap sick
4,Do you know how to speak english,know speak english


In [273]:
tokenizer = Tokenizer(num_words=None, split=" ")
tokenizer.fit_on_texts(x_feature['body'].values)

X = tokenizer.texts_to_sequences(x_feature['body'].values)
#print(X)
X = pad_sequences(X) # padding our text vector so they all have the same length
X[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,  473,  125,   31, 1141],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,  427, 4182,   67, 1592,
        1183, 7394,  863,  804,   65],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  409,  597,
        2327,  216, 5245, 2763,   18],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, 5246,  143, 1226,  805],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,   48,  284, 1434]])

In [274]:
index=tokenizer.word_index
len(index)

17030

In [275]:
y_feature['label1']=y_feature['label'].apply(lambda x:clean_text1(x))

In [276]:
embedding_matrix = np.zeros((len(index)+1, 300))
for word, i in tqdm(index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|█████████████████████████████████████████████████████████████████████████| 17030/17030 [00:00<00:00, 44994.19it/s]


In [277]:
model = Sequential()

model.add(Embedding(len(index) + 1,300,weights=[embedding_matrix],input_length=1000,trainable=False))
model.add(Bidirectional(LSTM(100, dropout=0.2,recurrent_dropout=0.2)))
#model.add(Bidirectional(LSTM(200, dropout=0.3, recurrent_dropout=0.2)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1000, 300)         5109300   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200)               320800    
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 603       
Total params: 5,430,703
Trainable params: 321,403
Non-trainable params: 5,109,300
_________________________________________________________________


In [278]:
y = pd.get_dummies(y_feature['label1']).values
[print(y_feature['label1'][i], y[i]) for i in range(0,5)]

negative [1 0 0]
negative [1 0 0]
neutral [0 1 0]
negative [1 0 0]
positive [0 0 1]


[None, None, None, None, None]

In [279]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [280]:
model.output_shape

(None, 3)

In [281]:
from keras.callbacks import EarlyStopping
batch_size = 32
epochs = 6
#es_callback = EarlyStopping(monitor='val_loss', patience=3,restore_best_weights=True)

#model.fit(X_train,y_train, callbacks=[es_callback],verbose=1)
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/6


186/559 [========>.....................] - ETA: 0s - loss: 1.1170 - accuracy: 0.28 - ETA: 1:00 - loss: 1.1028 - accuracy: 0.34 - ETA: 1:18 - loss: 1.0616 - accuracy: 0.41 - ETA: 1:25 - loss: 1.0586 - accuracy: 0.40 - ETA: 1:30 - loss: 1.0672 - accuracy: 0.40 - ETA: 1:34 - loss: 1.0575 - accuracy: 0.41 - ETA: 1:38 - loss: 1.0355 - accuracy: 0.43 - ETA: 1:39 - loss: 1.0137 - accuracy: 0.45 - ETA: 1:40 - loss: 0.9988 - accuracy: 0.46 - ETA: 1:40 - loss: 0.9992 - accuracy: 0.46 - ETA: 1:41 - loss: 0.9861 - accuracy: 0.47 - ETA: 1:43 - loss: 0.9657 - accuracy: 0.48 - ETA: 1:45 - loss: 0.9447 - accuracy: 0.50 - ETA: 1:45 - loss: 0.9407 - accuracy: 0.50 - ETA: 1:45 - loss: 0.9246 - accuracy: 0.51 - ETA: 1:46 - loss: 0.9191 - accuracy: 0.51 - ETA: 1:46 - loss: 0.9176 - accuracy: 0.52 - ETA: 1:46 - loss: 0.9116 - accuracy: 0.52 - ETA: 1:46 - loss: 0.9117 - accuracy: 0.52 - ETA: 1:46 - loss: 0.9091 - accuracy: 0.53 - ETA: 1:47 - loss: 0.9022 - accuracy: 0.54 - ETA: 1:47 - loss: 0.8985 - accuracy

372/559 [==================>...........] - ETA: 1:15 - loss: 0.6979 - accuracy: 0.70 - ETA: 1:15 - loss: 0.6969 - accuracy: 0.70 - ETA: 1:15 - loss: 0.6966 - accuracy: 0.70 - ETA: 1:15 - loss: 0.6953 - accuracy: 0.71 - ETA: 1:15 - loss: 0.6950 - accuracy: 0.71 - ETA: 1:14 - loss: 0.6944 - accuracy: 0.71 - ETA: 1:14 - loss: 0.6940 - accuracy: 0.71 - ETA: 1:14 - loss: 0.6928 - accuracy: 0.71 - ETA: 1:14 - loss: 0.6924 - accuracy: 0.71 - ETA: 1:13 - loss: 0.6913 - accuracy: 0.71 - ETA: 1:13 - loss: 0.6921 - accuracy: 0.71 - ETA: 1:13 - loss: 0.6914 - accuracy: 0.71 - ETA: 1:13 - loss: 0.6906 - accuracy: 0.71 - ETA: 1:13 - loss: 0.6884 - accuracy: 0.71 - ETA: 1:12 - loss: 0.6868 - accuracy: 0.71 - ETA: 1:12 - loss: 0.6857 - accuracy: 0.71 - ETA: 1:12 - loss: 0.6846 - accuracy: 0.71 - ETA: 1:12 - loss: 0.6841 - accuracy: 0.71 - ETA: 1:12 - loss: 0.6834 - accuracy: 0.71 - ETA: 1:11 - loss: 0.6823 - accuracy: 0.71 - ETA: 1:11 - loss: 0.6823 - accuracy: 0.71 - ETA: 1:11 - loss: 0.6833 - accura

559/559 [==============================] - ETA: 37s - loss: 0.6388 - accuracy: 0.738 - ETA: 37s - loss: 0.6391 - accuracy: 0.738 - ETA: 37s - loss: 0.6386 - accuracy: 0.739 - ETA: 36s - loss: 0.6382 - accuracy: 0.739 - ETA: 36s - loss: 0.6378 - accuracy: 0.739 - ETA: 36s - loss: 0.6373 - accuracy: 0.739 - ETA: 36s - loss: 0.6370 - accuracy: 0.739 - ETA: 36s - loss: 0.6365 - accuracy: 0.739 - ETA: 35s - loss: 0.6363 - accuracy: 0.739 - ETA: 35s - loss: 0.6357 - accuracy: 0.740 - ETA: 35s - loss: 0.6354 - accuracy: 0.740 - ETA: 35s - loss: 0.6355 - accuracy: 0.740 - ETA: 35s - loss: 0.6353 - accuracy: 0.740 - ETA: 34s - loss: 0.6365 - accuracy: 0.739 - ETA: 34s - loss: 0.6358 - accuracy: 0.739 - ETA: 34s - loss: 0.6359 - accuracy: 0.739 - ETA: 34s - loss: 0.6366 - accuracy: 0.739 - ETA: 34s - loss: 0.6365 - accuracy: 0.739 - ETA: 33s - loss: 0.6361 - accuracy: 0.740 - ETA: 33s - loss: 0.6358 - accuracy: 0.740 - ETA: 33s - loss: 0.6358 - accuracy: 0.740 - ETA: 33s - loss: 0.6358 - accurac

Epoch 2/6


...

559/559 [==============================] - ETA: 39s - loss: 0.4059 - accuracy: 0.842 - ETA: 39s - loss: 0.4055 - accuracy: 0.842 - ETA: 38s - loss: 0.4057 - accuracy: 0.842 - ETA: 38s - loss: 0.4056 - accuracy: 0.842 - ETA: 38s - loss: 0.4056 - accuracy: 0.842 - ETA: 38s - loss: 0.4055 - accuracy: 0.842 - ETA: 38s - loss: 0.4059 - accuracy: 0.842 - ETA: 37s - loss: 0.4057 - accuracy: 0.842 - ETA: 37s - loss: 0.4057 - accuracy: 0.842 - ETA: 37s - loss: 0.4053 - accuracy: 0.842 - ETA: 37s - loss: 0.4056 - accuracy: 0.842 - ETA: 37s - loss: 0.4054 - accuracy: 0.842 - ETA: 36s - loss: 0.4055 - accuracy: 0.841 - ETA: 36s - loss: 0.4058 - accuracy: 0.841 - ETA: 36s - loss: 0.4059 - accuracy: 0.841 - ETA: 36s - loss: 0.4060 - accuracy: 0.841 - ETA: 35s - loss: 0.4061 - accuracy: 0.841 - ETA: 35s - loss: 0.4066 - accuracy: 0.841 - ETA: 35s - loss: 0.4068 - accuracy: 0.841 - ETA: 35s - loss: 0.4067 - accuracy: 0.841 - ETA: 35s - loss: 0.4066 - accuracy: 0.841 - ETA: 34s - loss: 0.4062 - accurac

Epoch 6/6


186/559 [========>.....................] - ETA: 0s - loss: 0.3433 - accuracy: 0.84 - ETA: 58s - loss: 0.3282 - accuracy: 0.859 - ETA: 1:18 - loss: 0.3389 - accuracy: 0.86 - ETA: 1:27 - loss: 0.3359 - accuracy: 0.87 - ETA: 1:32 - loss: 0.3439 - accuracy: 0.87 - ETA: 1:38 - loss: 0.3229 - accuracy: 0.87 - ETA: 1:40 - loss: 0.3089 - accuracy: 0.87 - ETA: 1:42 - loss: 0.3173 - accuracy: 0.86 - ETA: 1:43 - loss: 0.3033 - accuracy: 0.86 - ETA: 1:45 - loss: 0.3172 - accuracy: 0.86 - ETA: 1:46 - loss: 0.3328 - accuracy: 0.85 - ETA: 1:46 - loss: 0.3335 - accuracy: 0.85 - ETA: 1:47 - loss: 0.3418 - accuracy: 0.85 - ETA: 1:47 - loss: 0.3474 - accuracy: 0.85 - ETA: 1:48 - loss: 0.3381 - accuracy: 0.86 - ETA: 1:49 - loss: 0.3253 - accuracy: 0.87 - ETA: 1:50 - loss: 0.3207 - accuracy: 0.87 - ETA: 1:50 - loss: 0.3232 - accuracy: 0.87 - ETA: 1:49 - loss: 0.3252 - accuracy: 0.87 - ETA: 1:51 - loss: 0.3267 - accuracy: 0.87 - ETA: 1:51 - loss: 0.3274 - accuracy: 0.87 - ETA: 1:50 - loss: 0.3249 - accuracy

372/559 [==================>...........] - ETA: 1:18 - loss: 0.3497 - accuracy: 0.86 - ETA: 1:18 - loss: 0.3506 - accuracy: 0.86 - ETA: 1:18 - loss: 0.3509 - accuracy: 0.86 - ETA: 1:17 - loss: 0.3508 - accuracy: 0.86 - ETA: 1:17 - loss: 0.3514 - accuracy: 0.86 - ETA: 1:17 - loss: 0.3526 - accuracy: 0.86 - ETA: 1:17 - loss: 0.3521 - accuracy: 0.86 - ETA: 1:17 - loss: 0.3529 - accuracy: 0.86 - ETA: 1:16 - loss: 0.3522 - accuracy: 0.86 - ETA: 1:16 - loss: 0.3521 - accuracy: 0.86 - ETA: 1:16 - loss: 0.3511 - accuracy: 0.86 - ETA: 1:16 - loss: 0.3515 - accuracy: 0.86 - ETA: 1:15 - loss: 0.3516 - accuracy: 0.86 - ETA: 1:15 - loss: 0.3513 - accuracy: 0.86 - ETA: 1:15 - loss: 0.3513 - accuracy: 0.86 - ETA: 1:15 - loss: 0.3515 - accuracy: 0.86 - ETA: 1:15 - loss: 0.3520 - accuracy: 0.86 - ETA: 1:14 - loss: 0.3523 - accuracy: 0.86 - ETA: 1:14 - loss: 0.3537 - accuracy: 0.86 - ETA: 1:14 - loss: 0.3546 - accuracy: 0.86 - ETA: 1:14 - loss: 0.3542 - accuracy: 0.86 - ETA: 1:13 - loss: 0.3551 - accura

559/559 [==============================] - ETA: 38s - loss: 0.3636 - accuracy: 0.856 - ETA: 38s - loss: 0.3636 - accuracy: 0.856 - ETA: 38s - loss: 0.3637 - accuracy: 0.856 - ETA: 37s - loss: 0.3636 - accuracy: 0.856 - ETA: 37s - loss: 0.3639 - accuracy: 0.856 - ETA: 37s - loss: 0.3635 - accuracy: 0.856 - ETA: 37s - loss: 0.3640 - accuracy: 0.856 - ETA: 37s - loss: 0.3638 - accuracy: 0.856 - ETA: 36s - loss: 0.3640 - accuracy: 0.856 - ETA: 36s - loss: 0.3637 - accuracy: 0.856 - ETA: 36s - loss: 0.3634 - accuracy: 0.856 - ETA: 36s - loss: 0.3633 - accuracy: 0.856 - ETA: 36s - loss: 0.3631 - accuracy: 0.856 - ETA: 35s - loss: 0.3629 - accuracy: 0.856 - ETA: 35s - loss: 0.3629 - accuracy: 0.856 - ETA: 35s - loss: 0.3628 - accuracy: 0.856 - ETA: 35s - loss: 0.3628 - accuracy: 0.856 - ETA: 35s - loss: 0.3633 - accuracy: 0.856 - ETA: 34s - loss: 0.3643 - accuracy: 0.856 - ETA: 34s - loss: 0.3646 - accuracy: 0.856 - ETA: 34s - loss: 0.3645 - accuracy: 0.856 - ETA: 34s - loss: 0.3646 - accurac

In [326]:
predictions = model.predict(X_test)
[print(x_feature['body'][i], predictions[i], y_test[i]) for i in range(0, 5)]

probably wont back honest [0.13139129 0.8627709  0.00583779] [0 1 0]
waited thirty minute seated although vacant table folk waiting [0.8664016  0.10569184 0.02790651] [1 0 0]
read star war try experiment borrow one [0.13093668 0.852275   0.01678836] [1 0 0]
hostel food crap sick [9.9854028e-01 1.3914889e-03 6.8221074e-05] [1 0 0]
know speak english [0.08232123 0.70845926 0.20921947] [0 1 0]


[None, None, None, None, None]

In [327]:
pos_count, neu_count, neg_count = 0, 0, 0
real_pos, real_neu, real_neg = 0, 0, 0
for i, prediction in enumerate(predictions):
    if np.argmax(prediction)==2:
        pos_count += 1
    elif np.argmax(prediction)==1:
        neu_count += 1
    else:
        neg_count += 1
    
    if np.argmax(y_test[i])==2:
        real_pos += 1
    elif np.argmax(y_test[i])==1:    
        real_neu += 1
    else:
        real_neg +=1

print('Positive predictions:', pos_count)
print('Neutral predictions:', neu_count)
print('Negative predictions:', neg_count)
print('Real positive:', real_pos)
print('Real neutral:', real_neu)
print('Real negative:', real_neg)

Positive predictions: 933
Neutral predictions: 1184
Negative predictions: 2348
Real positive: 1001
Real neutral: 1141
Real negative: 2323


In [348]:
tt=tokenizer.texts_to_sequences(['honest'])
tt

[[1141]]

In [344]:
tt=pad_sequences(tt)
tt

array([[ 7, 38]])

In [345]:
model.predict(tt)

array([[0.7608152 , 0.10702538, 0.13215938]], dtype=float32)

In [346]:
model.save('glove6.h5')

In [347]:
with open('pickle6.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [206]:
from keras.callbacks import EarlyStopping

In [ ]:
print